In [ ]:
!pip install -r requirements.txt --no-warn-conflicts --progress-bar emoji

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import girder_client
import json
import os
import pandas as pd
import psycopg2
import postgres_to_girder
import urllib
from datetime import date

In [32]:
activities_id, \
activities, \
config, \
context, \
girder_connection, \
postgres_connection, \
groups, \
postgres_tables, \
users = postgres_to_girder._main()

Connected to the Girder database 🏗🍃 and authenticated.
Connected to the Postgres database 🐘
Healthy Brain Network ― Example table survey (2018-02-21): number:
	5 rows, 4 columns
		rows: [{'text': 'Harold'}, {'text': 'Elizabeth'}, {'text': 'Mike'}, {'text': 'Arno'}, {'text': 'Jasmine'}]
		columns: [{'text': 'Share'}, {'text': 'Steal'}, {'text': 'Hoard'}, {'text': 'Violent'}]
Healthy Brain Network ― Example table survey (2018-02-21): image_sel:
	5 rows, 3 columns
		rows: [{'text': 'Harold'}, {'text': 'Elizabeth'}, {'text': 'Mike'}, {'text': 'Arno'}, {'text': 'Jasmine'}]
		columns: [{'text': '', 'image_url': 'https://firebasestorage.googleapis.com/v0/b/dataapp-3b8c4.appspot.com/o/images%2F1511319406548ahh-blue-merle-chihuahua--i-think-i-am-in-love-wallpaper-cuteimages.net.jpg?alt=media&token=6c5cb3cf-73f2-47e4-9578-0276bdf2541a'}, {'image_url': 'https://firebasestorage.googleapis.com/v0/b/dataapp-3b8c4.appspot.com/o/images%2F1511319425662Ig7IKQ9Q_400x400.jpg?alt=media&token=4b11a967-86c8-

In [ ]:
postgres_to_girder._delete_users(gc, "5b1830ddad97092bdb8e99d1")

In [ ]:
print({k for d in acts["act_data"] for k in json.loads(d)})

In [ ]:
print({d for d in acts["act_data"]})

In [9]:
{"questions":[{"title":"How did the children behave when given toys?","rows_count":5,"cols_count":4,"rows":[{"text":"Harold"},{"text":"Elizabeth"},{"text":"Mike"},{"text":"Arno"},{"text":"Jasmine"}],"cols":[{"text":"Share"},{"text":"Steal"},{"text":"Hoard"},{"text":"Violent"}],"type":"number"},{"title":"Which image do you like best?","rows_count":5,"cols_count":3,"rows":[{"text":"Harold"},{"text":"Elizabeth"},{"text":"Mike"},{"text":"Arno"},{"text":"Jasmine"}],"cols":[{"text":"","image_url":"https://firebasestorage.googleapis.com/v0/b/dataapp-3b8c4.appspot.com/o/images%2F1511319406548ahh-blue-merle-chihuahua--i-think-i-am-in-love-wallpaper-cuteimages.net.jpg?alt=media&token=6c5cb3cf-73f2-47e4-9578-0276bdf2541a"},{"image_url":"https://firebasestorage.googleapis.com/v0/b/dataapp-3b8c4.appspot.com/o/images%2F1511319425662Ig7IKQ9Q_400x400.jpg?alt=media&token=4b11a967-86c8-42b8-96f5-474073a3d4a0"},{"image_url":"https://firebasestorage.googleapis.com/v0/b/dataapp-3b8c4.appspot.com/o/images%2F1512069637555Screenshot%20from%202017-04-27%2010-38-05.png?alt=media&token=b8ab5bb5-03c2-4f79-9081-c352d8f8f9a3"}],"type":"image_sel"}],"answers":[],"frequency":"1d","instruction":"Please respond to the following questions.","accordion":True,"mode":"table"}

{'questions': [{'title': 'How did the children behave when given toys?',
   'rows_count': 5,
   'cols_count': 4,
   'rows': [{'text': 'Harold'},
    {'text': 'Elizabeth'},
    {'text': 'Mike'},
    {'text': 'Arno'},
    {'text': 'Jasmine'}],
   'cols': [{'text': 'Share'},
    {'text': 'Steal'},
    {'text': 'Hoard'},
    {'text': 'Violent'}],
   'type': 'number'},
  {'title': 'Which image do you like best?',
   'rows_count': 5,
   'cols_count': 3,
   'rows': [{'text': 'Harold'},
    {'text': 'Elizabeth'},
    {'text': 'Mike'},
    {'text': 'Arno'},
    {'text': 'Jasmine'}],
   'cols': [{'text': '',
     'image_url': 'https://firebasestorage.googleapis.com/v0/b/dataapp-3b8c4.appspot.com/o/images%2F1511319406548ahh-blue-merle-chihuahua--i-think-i-am-in-love-wallpaper-cuteimages.net.jpg?alt=media&token=6c5cb3cf-73f2-47e4-9578-0276bdf2541a'},
    {'image_url': 'https://firebasestorage.googleapis.com/v0/b/dataapp-3b8c4.appspot.com/o/images%2F1511319425662Ig7IKQ9Q_400x400.jpg?alt=media&token

In [7]:
postgres_to_girder._delete_users(girder_connection, ["5b1830ddad97092bdb8e99d1"])

,_accessLevel,_id,_modelType,admin,created,deleted,email,emailVerified,firstName,groupInvites,groups,lastName,login,public,size,status
0,2,5b1830ddad97092bdb8e99d1,user,True,2018-06-06T19:07:08.791000+00:00,False,jon.clucas@childmind.org,True,Jon,[],"[5b2d3884ad97092bdb530b2b, 5b2d38a0ad97092bdb5...",Clucas,jon.clucas,True,0,enabled
1,2,5b33c2cbad97094e44b91c2b,user,True,2018-06-27T17:00:59.684000+00:00,True,admin@mindlogger.org,False,Super,[],"[5b2d3884ad97092bdb530b2b, 5b2d38a0ad97092bdb5...",Admin,adminatmindlogger.org,True,0,enabled
2,2,5b33c2cdad97094e44b91c2f,user,False,2018-06-27T17:01:01.014000+00:00,True,admin@ab2cd.com,False,Admin,[],[5b2d38bcad97092bdb530b2e],Admin,adminatab2cd.com,True,0,enabled
3,2,5b33c2ccad97094e44b91c2e,user,False,2018-06-27T17:01:00.682000+00:00,True,arno@mindboggle.info,False,Mind,[],[5b2d38bcad97092bdb530b2e],Boggle,arnoatmindboggle.info,True,0,enabled
4,2,5b33c2ccad97094e44b91c2c,user,False,2018-06-27T17:01:00.038000+00:00,True,arno@binarybottle.com,False,Binary,[],[5b2d38b1ad97092bdb530b2d],Bottle,arnoatbinarybottle.com,True,0,enabled
5,2,5b33c2cdad97094e44b91c30,user,False,2018-06-27T17:01:01.343000+00:00,True,jon.clucas@chilldmind.org,False,Jon,[],[5b2d38bcad97092bdb530b2e],Clucas,jon.clucasatchilldmind.org,True,0,enabled
6,2,5b33c2cdad97094e44b91c31,user,False,2018-06-27T17:01:01.676000+00:00,True,jon@vasegurt.com,False,Jon,[],[5b2d38bcad97092bdb530b2e],Clucas,jonatvasegurt.com,True,0,enabled
7,2,5b33c2ccad97094e44b91c2d,user,False,2018-06-27T17:01:00.359000+00:00,True,arno@childmind.org,False,Arno,[],[5b2d38bcad97092bdb530b2e],Klein,arnoatchildmind.org,True,0,enabled
8,2,5b33c2cead97094e44b91c32,user,False,2018-06-27T17:01:02.003000+00:00,True,anirudh.krishnakumar@childmind.org,False,Anirudh,[],[5b2d38a0ad97092bdb530b2c],Krishnakumar,anirudh.krishnakumaratchildmind.org,True,0,enabled
9,2,5b33c2aaad97094e44b91c2a,user,False,2018-06-27T17:00:26.117000+00:00,True,erik2@test.com,False,Erik,[],"[5b2d3884ad97092bdb530b2b, 5b2d38a0ad97092bdb5...",P,erik2attest.com,True,0,enabled


In [13]:
rows= [{'text': 'Good question'}, {'text': 'Bad'}]
columns= [{'text': '1'}, {'text': '2'}]

In [14]:
rows

[{'text': 'Good question'}, {'text': 'Bad'}]

In [50]:
def table_options_from_postgres(
    rows,
    columns,
    response_type
):
    """
    Function to convert Postgres table options 
    encoded as rows and columns to dictionaries
    with (row, column) keys where 0 represents a
    header in either dimension and internal cells
    are 1-indexed.
    
    Parameters
    ----------
    rows: list
        rows[]: dictionary
            key: string
                "text" or "img_url"
            value: string
                row header
    
    columns: list
        columns[] dictionary
            key: string
                "text" or "img_url"
            value: string
                internal cell option or column header
    
    response_type: string

    Returns
    -------
    table: dictionary
        key: 2-tuple
            (row_number, column_number)
            row_number: int
                0 == header
            column_number: int
                0 == header
        value: dictionary
            key: string
                "text" or "img_url"
            value: string
                internal cell option or column header
                
    Examples
    --------
    >>> [
    ...     key for key in table_options_from_postgres(
    ...         rows=[
    ...             {'text': 'Good question'},
    ...             {'text': 'Bad'}
    ...         ],
    ...         columns=[
    ...             {'text': '1'},
    ...             {'text': '2'}
    ...         ],
    ...         response_type="image_sel"
    ...     )
    ... ]
    [(1, 0), (2, 0), (1, 1), (1, 2), (2, 1), (2, 2)]
    >>> [
    ...     key for key in table_options_from_postgres(
    ...         rows=[
    ...             {'text': 'Good question'},
    ...             {'text': 'Bad'}
    ...         ],
    ...         columns=[
    ...             {'text': '1'},
    ...             {'text': '2'}
    ...         ],
    ...         response_type="single_sel"
    ...     )
    ... ]
    [(1, 0), (2, 0), (0, 1), (0, 2)]
    """
    return(
        {
            **{
                (i+1,0): rows[i] for i in range(
                    len(rows)
                )
            },
            **{
                (
                    i+1 if response_type in {
                        "image_sel"
                    } else 0,
                    j+1
                ): columns[j] for i in range(
                    len(rows)
                ) for j in range(
                    len(columns)
                )
            } 
        }
    )

In [52]:
[k for k in table_options_from_postgres(
    rows,
    columns,
    "sing"
)]

[(1, 0), (2, 0), (0, 1), (0, 2)]